## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/MyDrive/ml-project-2-ogpteam2/

/content/gdrive/.shortcut-targets-by-id/1Gl--elA2yI5JHne6d00NJiWZ2VMJHQZc/ml-project-2-ogpteam2


In [26]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import tensorflow.python.platform
import tensorflow as tf
from data_loading import *
from data_handling import *
from data_prep import *
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Loading

In [27]:
# Load data and labels (images and groundtruth)
data_dir = 'Data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

TRAINING_SIZE = 20 # number of images to be used

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAINING_SIZE)
train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)


Loading Data/training/images/satImage_001.png
Loading Data/training/images/satImage_002.png
Loading Data/training/images/satImage_003.png
Loading Data/training/images/satImage_004.png
Loading Data/training/images/satImage_005.png
Loading Data/training/images/satImage_006.png
Loading Data/training/images/satImage_007.png
Loading Data/training/images/satImage_008.png
Loading Data/training/images/satImage_009.png
Loading Data/training/images/satImage_010.png
Loading Data/training/images/satImage_011.png
Loading Data/training/images/satImage_012.png
Loading Data/training/images/satImage_013.png
Loading Data/training/images/satImage_014.png
Loading Data/training/images/satImage_015.png
Loading Data/training/images/satImage_016.png
Loading Data/training/images/satImage_017.png
Loading Data/training/images/satImage_018.png
Loading Data/training/images/satImage_019.png
Loading Data/training/images/satImage_020.png
Loading Data/training/groundtruth/satImage_001.png
Loading Data/training/groundt

## Data Preprocessing

In [31]:
c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 9450 c1 = 3050


We have an imbalanced dataset, so we consider both under-sampling and over-sampling

In [34]:
# Oversampling
new_train_data, new_train_labels = oversample(train_data, train_labels, index_of_1s, index_of_0s)

# Undersampling
#new_train_data, new_train_labels = undersample(train_data, train_labels)

In [35]:
c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(new_train_labels)):
    if new_train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 3050 c1 = 3050
